In [19]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

nola = pd.read_csv("Nola_all_clean.csv")
nola['game_date'] = pd.to_datetime(nola['game_date'])
nola=nola.sort_values(['game_date', 'at_bat_number'], ascending=[True, True])
nola.reset_index(inplace=True)
nola.drop(columns=['index'],  axis=1, inplace=True)

In [20]:
#Now make an interaction variable for balls and strikes. Higher favors pitcher.
nola["runner_pressure"] = (nola.on_1b+2*nola.on_2b+3*nola.on_3b)/6.
nola["score_diff"] = nola.fld_score - nola.bat_score

In [21]:
#Average wOBA and launch speed angle calculated as average of all previous plate appearances for individual batter.
nola["woba_hist"] = None
nola["lsangle_hist"] = None
nola["prev_hth"] = 0

for bid in nola.batter.unique():
    hist = nola[nola.batter==bid][nola.woba_value>=0]
    while hist.shape[0] > 1:
        nola.woba_hist.iloc[hist.index[-1]] = hist[:-1].woba_value.mean()
        nola.lsangle_hist.iloc[hist.index[-1]] = hist[:-1].launch_speed_angle.mean()
        nola.prev_hth.iloc[hist.index[-1]] = hist.shape[0]-1
        hist.drop(hist.index[-1], inplace=True)

In [22]:
#backfill above categories for whole at-bat
x = nola.shape[0]-1
for ix, fill in enumerate(reversed(nola.woba_hist)):
    if ix==x:
        break
    if isinstance(fill, float):
        if nola.batter[x-ix] == nola.batter[x-ix-1]:
            nola.woba_hist[x-ix-1] = fill
            
for ix, fill in enumerate(reversed(nola.prev_hth)):
    if ix==x:
        break
    if nola.batter[x-ix] == nola.batter[x-ix-1]:
        nola.prev_hth[x-ix-1] = fill
        
for ix, fill in enumerate(reversed(nola.lsangle_hist)):
    if ix==x:
        break
    if isinstance(fill, float):
        if nola.batter[x-ix] == nola.batter[x-ix-1]:
            nola.lsangle_hist[x-ix-1] = fill

#Delete noisy values for thin histories
for ix, p in enumerate(nola.prev_hth):
    if p < 5:
        nola.woba_hist[ix] = None
        nola.lsangle_hist[ix] = None

In [23]:
#Makes a string from strikes and balls
nola["count_id"] = None
for ix, (b,s) in enumerate(zip(nola.balls, nola.strikes)):
    nola.count_id[ix] = str(b)+"-"+str(s)
    
#Create pitch count & batter count.
nola["pitch_count"] = None
nola["batter_count"] = None
x=len(nola.game_pk)-1
i, b = 0, 1

for ix, pk in enumerate(nola.game_pk):
    i += 1
    
    nola.pitch_count[ix]=i
    nola.batter_count[ix]=b
    
    if ix==x:
        break
    if nola.batter[ix] != nola.batter[ix+1]:
        b += 1
    if pk != nola.game_pk[ix+1]:
        i=0
        b=1

In [24]:
#Last pitch and average pitch histories
nola["pitch_last"] = None

for ix, p in enumerate(nola.pitch_type):
    if ix == nola.shape[0]-1:
        break
        
    nola.pitch_last[ix+1] = p
    
    if nola.pitch_count[ix] == 2:
        nola.pitch_last[ix-1] = None

In [25]:
#Pitch thrown to individual batter based on all past pitches
nola["pitch_hist"] = None

for bid in nola.batter.unique():
    hist = nola[nola.batter==bid][["batter", "pitch_type"]]
    while hist.shape[0] > 1:
        ix = hist.index[-1] 
        hist.drop(hist.index[-1], inplace=True) #no leakage - don't calculate based on current pitch
        commonest_pitch = hist.groupby(["pitch_type"]).agg({"pitch_type": "count"}).pitch_type.sort_values(ascending=False).index[0]
        nola.pitch_hist.iloc[ix] = commonest_pitch

In [26]:
#Predict most common pitch by pitch number
nola["pitch_common"] = None

for ix, n in enumerate(nola.pitch_common):
    if n==1:
        nola.number_pitch[ix]="FF"
    else:
        nola.number_pitch[ix]="CU"

In [88]:
count_df=nola.groupby(["count_id", "pitch_type"]).agg({'count_id': 'count'})
#df = count_df[4:8]==count_df[4:8].max()
count_df

count_id
count_id pitch_type          
0-0      CH               435
         CU              1032
         FF              1249
         SI               730
0-1      CH               314
         CU               609
         FF               567
         SI               346
0-2      CH                99
         CU               426
         FF               300
         SI               141
1-0      CH               281
         CU               322
         FF               310
         SI               304
1-1      CH               343
         CU               405
         FF               402
         SI               263
1-2      CH               190
         CU               548
         FF               356
         SI               230
2-0      CH                90
         CU                51
         FF               112
         SI               141
2-1      CH               160
         CU               166
         FF               176
         SI               158
2-2      CH               188
         CU               366
         FF               251
         SI               179
3-0      CH                 8
         CU                 1
         FF                57
         SI                57
3-1      CH                57
         CU                32
         FF                87
         SI                99
3-2      CH                91
         CU               180
         FF               148
         SI               113

In [89]:
#Predict most common pitch by pitch count (Nola only)
nola["count_pitch"] = None

for ix, count in enumerate(nola.count_id):
    if count in ["0-0", "2-1", "3-0"]:
        nola.count_pitch[ix]="FF"
    elif count in ["2-0", "3-1"]:
        nola.count_pitch[ix]="SI"
    else:
        nola.count_pitch[ix]="CU"

In [57]:
nola.drop(columns=['woba_value', 'launch_speed_angle', 'fld_score', 'bat_score', 'on_3b', 'on_2b', 'on_1b'],  axis=1, inplace=True)

In [58]:
#Last 3 pitches velo. gradient
from scipy.stats import linregress
nola["velo_grad"] = None
x = [1,2,3]

for ix, _ in enumerate(nola.release_speed):
    if nola.pitch_count[ix] > 3:
        y = nola.release_speed[ix-3:ix]
        nola.velo_grad[ix] = linregress(x,y)[0]

In [59]:
#Pitch results must be dropped to prevent LEAKAGE
nola.drop(columns=['release_speed', 'zone'],  axis=1, inplace=True)

In [262]:
nola.to_csv('Nola_feat.csv', index=False)

### All features created. Prepare for models (dummy variables)

In [68]:
nola.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13170 entries, 0 to 13169
Data columns (total 31 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   pitch_type       13170 non-null  int64  
 1   batter           13170 non-null  int64  
 2   pitch_number     13170 non-null  int64  
 3   inning           13170 non-null  int64  
 4   balls            13170 non-null  int64  
 5   strikes          13170 non-null  int64  
 6   runner_pressure  13170 non-null  float64
 7   score_diff       13170 non-null  int64  
 8   woba_hist        13170 non-null  float64
 9   lsangle_hist     13170 non-null  float64
 10  prev_hth         13170 non-null  int64  
 11  count_id         13170 non-null  object 
 12  pitch_count      13170 non-null  int64  
 13  batter_count     13170 non-null  int64  
 14  pitch_last       13170 non-null  int64  
 15  pitch_hist       13170 non-null  int64  
 16  number_pitch     13170 non-null  int64  
 17  count_pitch 

In [61]:
nola.tail()

,pitch_type,batter,pitch_number,inning,stand,balls,strikes,outs_when_up,game_pk,pitch_pk,...,lsangle_hist,prev_hth,count_id,pitch_count,batter_count,pitch_last,pitch_hist,number_pitch,count_pitch,velo_grad
13165,CU,664040,4,4,1,1,2,2,630945,2020092703604,...,None,2,1-2,89,20,CU,CU,CU,CU,-2.85
13166,FF,664040,5,4,1,1,2,2,630945,2020092703605,...,None,2,1-2,90,20,CU,CU,CU,CU,-3.0
13167,CH,664040,6,4,1,1,2,2,630945,2020092703606,...,None,2,1-2,91,20,FF,CU,CU,CU,7.25
13168,FF,664040,7,4,1,2,2,2,630945,2020092703607,...,None,2,2-2,92,20,CH,CU,CU,CU,2.7
13169,CU,664040,8,4,1,3,2,2,630945,2020092703608,...,None,2,3-2,93,20,FF,CU,CU,CU,-0.8


In [62]:
velo_mean = np.mean(nola.velo_grad)
woba_mean = np.mean(nola.woba_hist)
lsangle_mean = np.mean(nola.lsangle_hist)

In [63]:
values = {"woba_hist":woba_mean, "lsangle_hist":lsangle_mean,
          "pitch_last": "CU", "pitch_hist":"CU", "velo_grad":velo_mean}
nola.fillna(value=values, inplace=True)

In [64]:
nola.drop(columns=['outs_when_up','opponent',
                   'game_pk', 'pitch_pk', 'stand'],  axis=1, inplace=True)

In [65]:
#Dummyize counts
nola=pd.concat([nola,pd.get_dummies(nola.count_id)], axis=1)

In [66]:
#Multinomial only
nola=nola.replace("FF",0)
nola=nola.replace("SI",1)
nola=nola.replace("CU",2)
nola=nola.replace("CH",3)

In [38]:
#Binomial
nola=nola.replace("FF",0)
nola=nola.replace("SI",0)
nola=nola.replace("CU",1)
nola=nola.replace("CH",1)

In [67]:
nola.to_csv('Nola_multinomial.csv', index=False)